In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import statsmodels.api as sm
import scipy.stats as stats
import re
import warnings

from sklearn.exceptions import ConvergenceWarning
from scipy.stats import chi2_contingency
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import auc, classification_report, fbeta_score, make_scorer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier, LinearRegression, Ridge, Lasso
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, make_scorer, mean_absolute_error, mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import kagglehub
import tqdm as notebook_tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beert\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
final_model = pd.read_csv("data_preprocessed.csv")

In [33]:
final_model.describe()

,Rating,min_salary,max_salary,avg_salary,Company Age,Seniority_code,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,Easy_Apply
count,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000,2253.000000
mean,3.729892,54.267199,89.984021,72.125610,39.247670,1.204616,0.595206,0.734576,0.500666,0.526853,0.634709,0.786951,4.193076,0.608966,0.035508
std,0.643852,19.571034,29.310016,23.595818,42.024827,0.602935,0.490961,0.441657,0.500111,0.499389,0.481618,0.409553,9.478334,1.148112,0.185102
min,1.000000,24.000000,38.000000,33.500000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,3.314523,41.000000,70.000000,58.000000,17.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
50%,3.700000,50.000000,87.000000,69.000000,24.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,4.100000,64.000000,104.000000,80.500000,40.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,0.000000,0.000000
max,5.000000,113.000000,190.000000,150.000000,343.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,58.000000,4.000000,1.000000


## 3. MÈTRIQUES DE SELECCIÓ

En aquest projecte de predicció de sous, no hem d'escollir una mètrica de selecció concreta. Farem servir mètriques que analitzin l'error comès a les prediccions i aixi poder escollir el model que cometi menys error, és a dir, el que predigui millor el sou de les ofertes de treball.

In [34]:
X = final_model.drop(columns=['avg_salary', 'min_salary', 'max_salary'])
y = final_model[['avg_salary','min_salary','max_salary']]

y_avg = y['avg_salary']
y_min = y['min_salary']
y_max = y['max_salary']

Primer comprovem com funciona una regressió lineal per fer-nos una idea general del nostre model.

In [41]:
regr = make_pipeline(MinMaxScaler(), LinearRegression())

regr.fit(X, y_avg)

y_pred = regr.predict(X)

Per avaluar el rendiment dels models de predicció de salaris farem servir les mètriques: 
- MAE: indica l’error mitjà absolut i permet interpretar directament quants diners s’equivoca el model de mitjana del sou.
- RMSE: penalitza més els errors grans i és especialment útil quan volem evitar prediccions molt allunyades del valor real.
- R²: mesura quina proporció de la variabilitat del salari és capaç d’explicar el model.

Considerem que una combinació de MAE i RMSE és la més adequada per comparar models, ja que permet quantificar l’error real i, alhora, detectar si el model comet prediccions extremes, mentre que el R² serveix de suport per entendre el grau d’ajust global.

In [42]:
print('MAE:',mean_absolute_error(y_avg, y_pred))
print('MSE:',mean_squared_error(y_avg, y_pred))
print('R2:',r2_score(y_avg, y_pred))

MAE: 13.950105727094666
MSE: 338.4528003652989
R2: 0.39183582118536797


## 4. MODELITZACIÓ

**4.1 Salary mínim**

Primer cal dividir les dades en conjunt d’entrenament i test (tipus 80%-20% o 70%-30%):

In [44]:
X_train_lower, X_test_lower, y_train_lower, y_test_lower = \
    train_test_split(final_model,final_model['min_salary'].values,test_size=0.3, random_state=0)


In [47]:
y_train_lower.mean(),y_test_lower.mean()

(54.6563094483196, 53.3594674556213)

**Normalització de dades**


Normalitzem les dades una vegada fet el split del conjunt d'entrenament i de test per minimitzar la pèrdua d'informació.

In [48]:
# Definir les columnes numèriques contínues a normalitzar
num_cols = ['Company Age',
            'Company Offers', 'num_competitors']

scaler = StandardScaler()
X_train_lower[num_cols] = scaler.fit_transform(X_train_lower[num_cols])

# Comprovació
X_train_lower.describe()

,Rating,min_salary,max_salary,avg_salary,Company Age,Seniority_code,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,Easy_Apply
count,1577.000000,1577.000000,1577.000000,1577.000000,1.577000e+03,1577.000000,1577.000000,1577.000000,1577.000000,1577.000000,1577.000000,1577.000000,1.577000e+03,1.577000e+03,1577.000000
mean,3.708784,54.656309,90.470514,72.563412,-2.478114e-17,1.214331,0.593532,0.739379,0.509829,0.528852,0.642993,0.788840,-6.420567e-17,5.181510e-17,0.034242
std,0.637200,19.470818,29.156053,23.441865,1.000317e+00,0.602215,0.491330,0.439113,0.500062,0.499325,0.479269,0.408261,1.000317e+00,1.000317e+00,0.181908
min,1.000000,24.000000,38.000000,33.500000,-7.988518e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.353498e-01,-5.459822e-01,0.000000
25%,3.300000,42.000000,72.000000,59.000000,-5.412262e-01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-3.353498e-01,-5.459822e-01,0.000000
50%,3.700000,51.000000,87.000000,69.500000,-3.772826e-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-3.353498e-01,-5.459822e-01,0.000000
75%,4.100000,64.000000,104.000000,80.500000,4.428660e-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.161388e-01,-5.459822e-01,0.000000
max,5.000000,113.000000,190.000000,150.000000,7.093861e+00,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.912165e+00,2.857235e+00,1.000000


**Selecció de models**

In [49]:
# models a provar
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'KNN': KNeighborsRegressor(n_neighbors=5)
}

# Configurar cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Funcions de scoring
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
r2_scorer = make_scorer(r2_score)

# resultats
results_list = []

# Iterar sobre models
for name, model in models.items():
    mae_scores = cross_val_score(model, X_train_lower, y_train_lower, cv=kf, scoring=mae_scorer)
    rmse_scores = cross_val_score(model, X_train_lower, y_train_lower, cv=kf, scoring=rmse_scorer)
    r2_scores = cross_val_score(model, X_train_lower, y_train_lower, cv=kf, scoring=r2_scorer)
    
    # Agregar a la lista
    results_list.append({
        'Model': name,
        'MAE_mean': -mae_scores.mean(),  # Negatiu perque sklearn retorna negatiu
        'MAE_std': mae_scores.std(),
        'RMSE_mean': -rmse_scores.mean(),
        'RMSE_std': rmse_scores.std(),
        'R2_mean': r2_scores.mean(),
        'R2_std': r2_scores.std()
    })

results = pd.DataFrame(results_list)


results = results.sort_values(by='MAE_mean')
print(results)

              Model      MAE_mean       MAE_std     RMSE_mean      RMSE_std  \
0  LinearRegression  3.094536e-14  1.094626e-14  4.084768e-14  1.440713e-14   
1             Ridge  7.045927e-05  2.782774e-06  9.768632e-05  7.714712e-06   
4  GradientBoosting  3.143319e-03  1.140883e-03  7.186783e-03  2.409070e-03   
2             Lasso  3.624082e-03  2.246688e-04  4.637841e-03  3.559909e-04   
3      RandomForest  5.200321e-03  1.851323e-03  3.882302e-02  9.074716e-03   
5               KNN  7.278722e-02  1.346828e-02  2.440572e-01  4.955980e-02   

    R2_mean        R2_std  
0  1.000000  0.000000e+00  
1  1.000000  3.920538e-12  
4  1.000000  8.654048e-08  
2  1.000000  3.123518e-09  
3  0.999996  1.781770e-06  
5  0.999833  6.559223e-05  


**Selecció d'hiperparàmetres**

Definim el grid de paràmetres:

In [50]:
param_grids = {
    "KNN": {
        "model": KNeighborsRegressor(),
        "params": {
            "n_neighbors": [3, 5, 7, 9, 15],
            "weights": ["uniform", "distance"],
            "p": [1, 2]  # Manhattan o Euclidiana
        }
    },

    "RandomForest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200, 300],
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4]
        }
    },

    "GradientBoosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "params": {
            "n_estimators": [100, 200, 300],
            "learning_rate": [0.01, 0.05, 0.1],
            "max_depth": [2, 3, 4],
            "subsample": [0.6, 0.8, 1.0]
        }
    },

    "Ridge": {
        "model": Ridge(),
        "params": {
            "alpha": [0.01, 0.1, 1, 10, 100]
        }
    },

    "Lasso": {
        "model": Lasso(max_iter=5000),
        "params": {
            "alpha": [0.0001, 0.001, 0.01, 0.1, 1]
        }
    }
}

Executar gridsearch pel model:

In [51]:
best_models = {}

for name, cfg in param_grids.items():
    print(f"\n Optimitzant {name}...\n")
    
    grid = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1
    )

    grid.fit(X_train_lower, y_train_lower)

    best_models[name] = grid.best_estimator_

    print(f"Millor score MAE: {-grid.best_score_:.2f}")
    print(f"Millors hiperparàmetres: {grid.best_params_}")


 Optimitzant KNN...

Millor score MAE: 0.05
Millors hiperparàmetres: {'n_neighbors': 3, 'p': 2, 'weights': 'distance'}

 Optimitzant RandomForest...

Millor score MAE: 0.00
Millors hiperparàmetres: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}

 Optimitzant GradientBoosting...

Millor score MAE: 0.00
Millors hiperparàmetres: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300, 'subsample': 1.0}

 Optimitzant Ridge...

Millor score MAE: 0.00
Millors hiperparàmetres: {'alpha': 0.01}

 Optimitzant Lasso...

Millor score MAE: 0.00
Millors hiperparàmetres: {'alpha': 0.0001}


Veiem que alguns hiperparametres escolllits estant a un extrem del grid així que explorem un nou espai d'hiperparàmetres en aquells casos per veure si aconseguim millorar encara més el model.

In [52]:
param_grids_refined = {
    "KNN": {
        "model": KNeighborsRegressor(),
        "params": {
            "n_neighbors": [15, 20, 25, 30, 40, 50],
            "weights": ["uniform", "distance"],
            "p": [1, 2]
        }
    },

    "RandomForest": {
        "model": RandomForestRegressor(random_state=42),
        "params": {
            "n_estimators": [200, 300, 400],
            "max_depth": [5, 10, 15, 20],
            "min_samples_split": [2, 3, 4, 6],
            "min_samples_leaf": [2, 4, 6]
        }
    },

    "GradientBoosting": {
        "model": GradientBoostingRegressor(random_state=42),
        "params": {
            "n_estimators": [300, 400, 500],
            "learning_rate": [0.005, 0.01, 0.02],
            "max_depth": [4, 5, 6],
            "subsample": [0.6, 0.8, 1.0]
        }
    },

    "Ridge": {
        "model": Ridge(),
        "params": {
            "alpha": [100, 200, 300, 500, 800, 1000]
        }
    },

    "Lasso": {
        "model": Lasso(max_iter=8000),
        "params": {
            "alpha": [1, 2, 5, 10, 20, 50]
        }
    }
}

In [53]:
best_models_refined = {}

for name, cfg in param_grids_refined.items():
    print(f"\n Optimitzant {name}...\n")

    grid = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1,
    )

    grid.fit(X_train_lower, y_train_lower)

    best_models_refined[name] = grid.best_estimator_

    print(f"Millor MAE: {-grid.best_score_:.2f}")
    print(f"Millors hiperparàmetres: {grid.best_params_}")


 Optimitzant KNN...

Millor MAE: 0.22
Millors hiperparàmetres: {'n_neighbors': 15, 'p': 2, 'weights': 'distance'}

 Optimitzant RandomForest...

Millor MAE: 0.01
Millors hiperparàmetres: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 200}

 Optimitzant GradientBoosting...

Millor MAE: 0.00
Millors hiperparàmetres: {'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 500, 'subsample': 1.0}

 Optimitzant Ridge...

Millor MAE: 0.01
Millors hiperparàmetres: {'alpha': 100}

 Optimitzant Lasso...

Millor MAE: 0.04
Millors hiperparàmetres: {'alpha': 1}


In [54]:
def get_mae(model):
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
    scores = cross_val_score(model, X_train_lower, y_train_lower, cv=5, scoring=mae_scorer)
    return -scores.mean()

rows = []

# Afegir models de la 1a optimització
for name, model in best_models.items():
    rows.append({
        "Model": f"{name} (Round 1)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })

# Afegir models de la 2a optimització
for name, model in best_models_refined.items():
    rows.append({
        "Model": f"{name} (Round 2)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })

results_table = pd.DataFrame(rows)
results_table

,Model,MAE,Hiperparàmetres
0,KNN (Round 1),5.143520e-02,"{'algorithm': 'auto', 'leaf_size': 30, 'metric..."
1,RandomForest (Round 1),4.579144e-03,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
2,GradientBoosting (Round 1),7.702125e-06,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
3,Ridge (Round 1),7.415367e-07,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
4,Lasso (Round 1),1.091713e-04,"{'alpha': 0.0001, 'copy_X': True, 'fit_interce..."
5,KNN (Round 2),2.208509e-01,"{'algorithm': 'auto', 'leaf_size': 30, 'metric..."
6,RandomForest (Round 2),6.708910e-03,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
7,GradientBoosting (Round 2),6.447478e-04,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
8,Ridge (Round 2),5.718850e-03,"{'alpha': 100, 'copy_X': True, 'fit_intercept'..."
9,Lasso (Round 2),4.130604e-02,"{'alpha': 1, 'copy_X': True, 'fit_intercept': ..."


Veiem que el Ridge obté el MAE més baix.

Veiem que el segon model que presenta millors resultats és el Gradient Boosting Regressor però tenint certa distància amb el Random Forest que per tant és el que escollim com a millor model per predir el salari estimat.

A continuació amb aquest model entrenarem de manera final sobre tot el conjunt d'entrenament i després avaluarem sobre el conjunt de test.

In [55]:
model_entries = [r for r in rows]

model_results = [
    (entry["Model"], entry["MAE"], entry["Hiperparàmetres"])
    for entry in model_entries
]

# Seleccionar el model amb MAE més baix
best_model_name, best_model_mae, best_model_params = min(model_results, key=lambda x: x[1])

print("Millor model:", best_model_name)
print("MAE:", best_model_mae)
best_model_params

Millor model: Ridge (Round 1)
MAE: 7.415367062913272e-07


{'alpha': 0.01,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.0001}

**Entrenament model seleccionat**

Entrenarem el model unicament pel valor mig del salari i per verificar si és correcte, comprovarem si aquest valor es troba dins de l'interval real de salari mínim i màxim.

Fem això perquè el valor mig del salari resumeix de manera senzilla la informació salarial de cada empresa i ens permet entrenar el model amb una sola variable contínua, evitant la complexitat de tenir múltiples valors per cada catacterística. A més, verificar que el valor predit es troba dins de l’interval real de salari mínim i màxim ens assegura que el model respecta les restriccions reals del salari i evita prediccions fora de rang.

In [57]:
from sklearn.linear_model import Ridge
best_ridge = Ridge(
    alpha=0.01,
    copy_X=True,
    fit_intercept=True,
    max_iter=None,
    positive=False,
    random_state=None,
    solver='auto',
    tol=0.0001
)
best_ridge.fit(X_train_lower, y_train_lower)


,alpha,0.01
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


**Prova conjunt de test**

In [58]:
y_pred = best_ridge.predict(X_test_lower)

In [59]:
mae = mean_absolute_error(y_test_lower, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_lower, y_pred))
r2 = r2_score(y_test_lower, y_pred)

In [62]:
print("Resultats del model:")
print(f"MAE  : {mae}")
print(f"RMSE : {rmse}")
print(f"R²   : {r2}")

Resultats del model:
MAE  : 7.127426577666616e-07
RMSE : 1.0994597983011891e-06
R²   : 0.9999999999999969


**4.2 Max Salary**


Repetim tot el procediment per max salary:

In [63]:
X_train_high, X_test_high, y_train_high, y_test_high = \
    train_test_split(final_model,final_model['max_salary'].values,test_size=0.3, random_state=0)


In [64]:
y_train_high.mean(),y_test_high.mean()

(90.47051363348129, 88.8491124260355)

In [65]:
# Definir les columnes numèriques contínues a normalitzar
num_cols = ['Company Age',
            'Company Offers', 'num_competitors']

scaler = StandardScaler()
X_train_high[num_cols] = scaler.fit_transform(X_train_high[num_cols])

# Comprovació
X_train_high.describe()

,Rating,min_salary,max_salary,avg_salary,Company Age,Seniority_code,contains_requirements,contains_team,contains_reporting,contains_information,contains_management,contains_skills,Company Offers,num_competitors,Easy_Apply
count,1577.000000,1577.000000,1577.000000,1577.000000,1.577000e+03,1577.000000,1577.000000,1577.000000,1577.000000,1577.000000,1577.000000,1577.000000,1.577000e+03,1.577000e+03,1577.000000
mean,3.708784,54.656309,90.470514,72.563412,-2.478114e-17,1.214331,0.593532,0.739379,0.509829,0.528852,0.642993,0.788840,-6.420567e-17,5.181510e-17,0.034242
std,0.637200,19.470818,29.156053,23.441865,1.000317e+00,0.602215,0.491330,0.439113,0.500062,0.499325,0.479269,0.408261,1.000317e+00,1.000317e+00,0.181908
min,1.000000,24.000000,38.000000,33.500000,-7.988518e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.353498e-01,-5.459822e-01,0.000000
25%,3.300000,42.000000,72.000000,59.000000,-5.412262e-01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-3.353498e-01,-5.459822e-01,0.000000
50%,3.700000,51.000000,87.000000,69.500000,-3.772826e-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-3.353498e-01,-5.459822e-01,0.000000
75%,4.100000,64.000000,104.000000,80.500000,4.428660e-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.161388e-01,-5.459822e-01,0.000000
max,5.000000,113.000000,190.000000,150.000000,7.093861e+00,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.912165e+00,2.857235e+00,1.000000


In [ ]:
# models a provar
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'Lasso': Lasso(alpha=0.1, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'KNN': KNeighborsRegressor(n_neighbors=5)
}

# Configurar cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Funcions de scoring
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
r2_scorer = make_scorer(r2_score)

# resultats
results_list = []

# Iterar sobre models
for name, model in models.items():
    mae_scores = cross_val_score(model, X_train_high, y_train_high, cv=kf, scoring=mae_scorer)
    rmse_scores = cross_val_score(model, X_train_high, y_train_high, cv=kf, scoring=rmse_scorer)
    r2_scores = cross_val_score(model, X_train_high, y_train_high, cv=kf, scoring=r2_scorer)
    
    # Agregar a la lista
    results_list.append({
        'Model': name,
        'MAE_mean': -mae_scores.mean(),  # Negatiu perque sklearn retorna negatiu
        'MAE_std': mae_scores.std(),
        'RMSE_mean': -rmse_scores.mean(),
        'RMSE_std': rmse_scores.std(),
        'R2_mean': r2_scores.mean(),
        'R2_std': r2_scores.std()
    })

results = pd.DataFrame(results_list)


results = results.sort_values(by='MAE_mean')
print(results)

              Model      MAE_mean       MAE_std     RMSE_mean      RMSE_std  \
0  LinearRegression  3.094536e-14  1.094626e-14  4.084768e-14  1.440713e-14   
1             Ridge  7.045927e-05  2.782774e-06  9.768632e-05  7.714712e-06   
4  GradientBoosting  3.143319e-03  1.140883e-03  7.186783e-03  2.409070e-03   
2             Lasso  3.624082e-03  2.246688e-04  4.637841e-03  3.559909e-04   
3      RandomForest  5.200321e-03  1.851323e-03  3.882302e-02  9.074716e-03   
5               KNN  7.278722e-02  1.346828e-02  2.440572e-01  4.955980e-02   

    R2_mean        R2_std  
0  1.000000  0.000000e+00  
1  1.000000  3.920538e-12  
4  1.000000  8.654048e-08  
2  1.000000  3.123518e-09  
3  0.999996  1.781770e-06  
5  0.999833  6.559223e-05  


In [67]:
best_models = {}

for name, cfg in param_grids.items():
    print(f"\n Optimitzant {name}...\n")
    
    grid = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1
    )

    grid.fit(X_train_high, y_train_high)

    best_models[name] = grid.best_estimator_

    print(f"Millor score MAE: {-grid.best_score_:.2f}")
    print(f"Millors hiperparàmetres: {grid.best_params_}")


 Optimitzant KNN...

Millor score MAE: 0.06
Millors hiperparàmetres: {'n_neighbors': 3, 'p': 2, 'weights': 'distance'}

 Optimitzant RandomForest...

Millor score MAE: 0.01
Millors hiperparàmetres: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}

 Optimitzant GradientBoosting...

Millor score MAE: 0.00
Millors hiperparàmetres: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 300, 'subsample': 1.0}

 Optimitzant Ridge...

Millor score MAE: 0.00
Millors hiperparàmetres: {'alpha': 0.01}

 Optimitzant Lasso...

Millor score MAE: 0.00
Millors hiperparàmetres: {'alpha': 0.0001}


In [68]:
def get_mae2(model):
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
    scores = cross_val_score(model, X_train_high, y_train_high, cv=5, scoring=mae_scorer)
    return -scores.mean()

rows = []

# Afegir models de la 1a optimització
for name, model in best_models.items():
    rows.append({
        "Model": f"{name} (Round 1)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })

# Afegir models de la 2a optimització
for name, model in best_models_refined.items():
    rows.append({
        "Model": f"{name} (Round 2)",
        "MAE": get_mae(model),
        "Hiperparàmetres": model.get_params()
    })

results_table = pd.DataFrame(rows)
results_table

,Model,MAE,Hiperparàmetres
0,KNN (Round 1),5.143520e-02,"{'algorithm': 'auto', 'leaf_size': 30, 'metric..."
1,RandomForest (Round 1),5.173143e-03,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
2,GradientBoosting (Round 1),7.702125e-06,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
3,Ridge (Round 1),7.415367e-07,"{'alpha': 0.01, 'copy_X': True, 'fit_intercept..."
4,Lasso (Round 1),1.091713e-04,"{'alpha': 0.0001, 'copy_X': True, 'fit_interce..."
5,KNN (Round 2),2.208509e-01,"{'algorithm': 'auto', 'leaf_size': 30, 'metric..."
6,RandomForest (Round 2),6.708910e-03,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."
7,GradientBoosting (Round 2),6.447478e-04,"{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': ..."
8,Ridge (Round 2),5.718850e-03,"{'alpha': 100, 'copy_X': True, 'fit_intercept'..."
9,Lasso (Round 2),4.130604e-02,"{'alpha': 1, 'copy_X': True, 'fit_intercept': ..."


In [69]:
model_entries = [r for r in rows]

model_results = [
    (entry["Model"], entry["MAE"], entry["Hiperparàmetres"])
    for entry in model_entries
]

# Seleccionar el model amb MAE més baix
best_model_name, best_model_mae, best_model_params = min(model_results, key=lambda x: x[1])

print("Millor model:", best_model_name)
print("MAE:", best_model_mae)
best_model_params

Millor model: Ridge (Round 1)
MAE: 7.415367062913272e-07


{'alpha': 0.01,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.0001}

In [70]:
from sklearn.linear_model import Ridge
best_ridge = Ridge(
    alpha=0.01,
    copy_X=True,
    fit_intercept=True,
    max_iter=None,
    positive=False,
    random_state=None,
    solver='auto',
    tol=0.0001
)
best_ridge.fit(X_train_high, y_train_high)


,alpha,0.01
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [71]:
y_pred_high = best_ridge.predict(X_test_high)

In [72]:
mae = mean_absolute_error(y_test_high, y_pred_high)
rmse = np.sqrt(mean_squared_error(y_test_high, y_pred_high))
r2 = r2_score(y_test_high, y_pred_high)

In [73]:
print("Resultats del model:")
print(f"MAE  : {mae}")
print(f"RMSE : {rmse}")
print(f"R²   : {r2}")

Resultats del model:
MAE  : 6.520980165367671e-07
RMSE : 9.515204264895568e-07
R²   : 0.999999999999999


In [74]:
df_export = pd.DataFrame({
    "real_min_salary": y_test_lower,
    "pred_min_salary": y_pred,
    "real_max_salary": y_test_high,
    "pred_max_salary": y_pred_high
})

file_name = "data_predicted.csv"
df_export.to_csv(file_name, index=False)
df_export.head()


,real_min_salary,pred_min_salary,real_max_salary,pred_max_salary
0,50.0,49.999999,86.0,86.000001
1,42.0,42.000000,66.0,66.000000
2,37.0,37.000000,70.0,70.000000
3,24.0,24.000000,48.0,48.000000
4,110.0,110.000000,190.0,189.999999
